In [37]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [ ]:
#Leemos los CSVs

df1 = pd.read_csv('../files/Customer Flight Activity.csv')

In [ ]:
df2 = pd.read_csv('../files/Customer Loyalty History.csv')

Exploramos el primer CSV:

In [40]:
df1.head()

#Este df contiene num. de vuelos, distancia y puntos acumulados y usados, de cada usuario por mes y año.

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0
3,100214,2017,1,0,0,0,0,0.0,0,0
4,100272,2017,1,0,0,0,0,0.0,0,0


In [41]:
df1.shape

(405624, 10)

In [42]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB


Cambiar el tipo de dato de Points Accumulated a int

In [43]:
df1.isna().sum()

Loyalty Number                 0
Year                           0
Month                          0
Flights Booked                 0
Flights with Companions        0
Total Flights                  0
Distance                       0
Points Accumulated             0
Points Redeemed                0
Dollar Cost Points Redeemed    0
dtype: int64

In [44]:
df1.duplicated().sum()

1864

In [ ]:
#df1 = df1[df1['Total Flights'] != 0]

#Eliminamos valores que no aportan a nuestro df

#UPDATE: sí que pueden aportar en un momento dado, p.ej. para calcular una media de vuelo entre todos los meses de un mismo cliente

In [49]:
df1['Loyalty Number'].is_unique

False

In [50]:
df1['Loyalty Number'].value_counts()

Loyalty Number
678205    72
989528    48
373638    48
684889    48
684881    48
          ..
428526    24
428536    24
428565    24
428590    24
999891    24
Name: count, Length: 16737, dtype: int64

In [ ]:
#Tomamos un valor de 'Loyalty Number' aleatorio para ver qué información recoge
df1.loc[df1['Loyalty Number'] == 617489, :]

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
9750,617489,2017,1,6,0,6,2172,217.00,0,0
9751,617489,2017,1,5,3,8,2648,264.00,0,0
26651,617489,2017,2,3,0,3,861,86.00,0,0
26652,617489,2017,2,0,0,0,0,0.00,0,0
43552,617489,2017,3,7,0,7,3374,337.00,0,0
60453,617489,2017,4,2,0,2,1626,162.00,0,0
60454,617489,2017,4,2,2,4,1848,184.00,0,0
77354,617489,2017,5,13,0,13,1677,167.00,0,0
77355,617489,2017,5,6,0,6,2022,202.00,0,0
89764,617489,2018,11,11,1,12,2556,255.00,353,29


Como podemos observar, para un mismo cliente, año y mes hay varias filas, que pueden estar o no duplicadas.
Para ello, vamos a agrupar por número de cliente, año y mes, sumando el resto de columnas entre sí. 
Con esto, ya tendremos nuestro dataframe libre de duplicados.

In [ ]:
df1 = df1.groupby(["Loyalty Number", "Year", "Month"], as_index=False).sum()

In [ ]:
# Vemos los valores únicos para cada columna del df y el conteo de los mismos. Con esto comprobamos que los datos no contienen valores atípicos:

for columna in df1:
    print(f'Valores únicos de {columna}:')
    print(df1[columna].unique())
    print(f'Conteo de valores únicos de {columna}:')
    print(df1[columna].nunique())
    print('---'*30)

Valores únicos de Loyalty Number:
[100018 100102 100140 ... 999940 999982 999986]
Conteo de valores únicos de Loyalty Number:
16737
------------------------------------------------------------------------------------------
Valores únicos de Year:
[2017 2018]
Conteo de valores únicos de Year:
2
------------------------------------------------------------------------------------------
Valores únicos de Month:
[ 1  2  3  4  5  6  7  8  9 10 11 12]
Conteo de valores únicos de Month:
12
------------------------------------------------------------------------------------------
Valores únicos de Flights Booked:
[ 3  2 14  4  0 10  9  6 13  1  7  5 11 17 15  8 12 16 20 18 19 30 22 21
 25 32 24 26 31 35 23 27 36 28 34 39 37 38 29 33]
Conteo de valores únicos de Flights Booked:
40
------------------------------------------------------------------------------------------
Valores únicos de Flights with Companions:
[ 0  2  3  1  5  4  7  6  8  9 10 11 16 19 12 13 17 18 14 15 20]
Conteo de valores ú

In [54]:
#Ordenamos y comprobamos que no existen valores negativos:

columnas = ['Loyalty Number', 'Distance', 'Points Accumulated']
for columna in columnas:
    print(f'Valores ordenados de {columna}:')
    print(df1[columna].sort_values(ascending=True))
    print('---'*30)

Valores ordenados de Loyalty Number:
0         100018
23        100018
22        100018
21        100018
20        100018
           ...  
401665    999986
401664    999986
401686    999986
401674    999986
401687    999986
Name: Loyalty Number, Length: 401688, dtype: int64
------------------------------------------------------------------------------------------
Valores ordenados de Distance:
200843        0
221750        0
221751        0
221752        0
221753        0
          ...  
398406     9716
261618     9752
138066     9781
356514     9903
285810    11244
Name: Distance, Length: 401688, dtype: int64
------------------------------------------------------------------------------------------
Valores ordenados de Points Accumulated:
200843       0.0
221750       0.0
221751       0.0
221752       0.0
221753       0.0
           ...  
356514     990.0
99158     1009.5
223430    1012.5
285810    1124.0
44438     1216.5
Name: Points Accumulated, Length: 401688, dtype: float64
------

Comenzamos la exploración del segundo CSV:

In [55]:
df2.head()

#Este df contiene información personal de cada usuario, desde su info geográfica, hasta su educación, salario, situación personal, cancelaciones...

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN
4,530508,Canada,Quebec,Hull,J8Y 3Z5,Male,Bachelor,103495.0,Married,Star,3842.79,Standard,2014,10,NaN,NaN


In [56]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16737 entries, 0 to 16736
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Loyalty Number      16737 non-null  int64  
 1   Country             16737 non-null  object 
 2   Province            16737 non-null  object 
 3   City                16737 non-null  object 
 4   Postal Code         16737 non-null  object 
 5   Gender              16737 non-null  object 
 6   Education           16737 non-null  object 
 7   Salary              12499 non-null  float64
 8   Marital Status      16737 non-null  object 
 9   Loyalty Card        16737 non-null  object 
 10  CLV                 16737 non-null  float64
 11  Enrollment Type     16737 non-null  object 
 12  Enrollment Year     16737 non-null  int64  
 13  Enrollment Month    16737 non-null  int64  
 14  Cancellation Year   2067 non-null   float64
 15  Cancellation Month  2067 non-null   float64
dtypes: f

In [57]:
df2.shape

(16737, 16)

In [58]:
df2.columns

Index(['Loyalty Number', 'Country', 'Province', 'City', 'Postal Code',
       'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card',
       'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month',
       'Cancellation Year', 'Cancellation Month'],
      dtype='object')

In [59]:
df2.isna().sum()

Loyalty Number            0
Country                   0
Province                  0
City                      0
Postal Code               0
Gender                    0
Education                 0
Salary                 4238
Marital Status            0
Loyalty Card              0
CLV                       0
Enrollment Type           0
Enrollment Year           0
Enrollment Month          0
Cancellation Year     14670
Cancellation Month    14670
dtype: int64

Habría que imputar nulos en la columna Salary y cambiar el tipo de dato en Cancellation Year y Cancellation Month a tipo int.

In [60]:
df2.duplicated().sum()

0

In [61]:
df2['Loyalty Number'].is_unique

True

In [ ]:
# Vemos los valores únicos por cada columna y el conteo de los mismos:

for col in df2:
    print(f'Valores únicos de {col}:')
    print(df2[col].unique())
    print(f'Conteo de valores únicos de {col}:')
    print(df2[col].nunique())
    print('---'*30)

Valores únicos de Loyalty Number:
[480934 549612 429460 ... 776187 906428 652627]
Conteo de valores únicos de Loyalty Number:
16737
------------------------------------------------------------------------------------------
Valores únicos de Country:
['Canada']
Conteo de valores únicos de Country:
1
------------------------------------------------------------------------------------------
Valores únicos de Province:
['Ontario' 'Alberta' 'British Columbia' 'Quebec' 'Yukon' 'New Brunswick'
 'Manitoba' 'Nova Scotia' 'Saskatchewan' 'Newfoundland'
 'Prince Edward Island']
Conteo de valores únicos de Province:
11
------------------------------------------------------------------------------------------
Valores únicos de City:
['Toronto' 'Edmonton' 'Vancouver' 'Hull' 'Whitehorse' 'Trenton' 'Montreal'
 'Dawson Creek' 'Quebec City' 'Fredericton' 'Ottawa' 'Tremblant' 'Calgary'
 'Thunder Bay' 'Whistler' 'Peace River' 'Winnipeg' 'Sudbury'
 'West Vancouver' 'Halifax' 'London' 'Regina' 'Kelowna' "St.

Salary tiene valores negativos, pasar a positivo y cambiar a tipo int.

### Cambios a aplicar:

DF1:

- Cambiar el tipo de dato de Points Accumulated a int
- Agrupar por número de cliente, año y mes, sumando el resto de columnas entre sí. 

DF2:

- Imputar nulos en la columna Salary, pasar a positivo y cambiar a tipo int.
- Cambiar el tipo de dato en Cancellation Year y Cancellation Month a tipo int.